In [18]:
import os
import pandas as pd
import seaborn as sns
import numpy as np


%matplotlib inline

INPUT_FILE = os.path.join("input", "titanic.csv")
REPORT_FILE = os.path.join("output","report.html")
GENERATE_REPORT = False

raw_data = pd.read_csv(INPUT_FILE)
raw_data.head()

#from pandas_profiling import ProfileReport

#profile = ProfileReport(raw_data)
#profile.to_file(REPORT_FILE)


#Counting People in different classes
data = raw_data.copy()
data["alive"] = data["survived"].map({0: False, 1: True})
data["survived"].value_counts()

data["class"] = data["pclass"].astype(pd.CategoricalDtype(ordered=True))
data["class"] = data["class"].map({1: 'first', 2: 'second', 3: 'third'})

data["class"].value_counts(sort=False)

# Data manipulation of Cabin data
data["cabin"].value_counts()
data["deck"] = data["cabin"].str["0"]
data["deck"].value_counts()

x = "B96 B98"
x[0:3]



Series([], Name: deck, dtype: int64)